<a href="https://colab.research.google.com/github/ogut77/BA/blob/main/TTTwAutoML3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install seffaflik
!pip install pandasql
!pip install  holidays



In [ ]:
%%time
lh=240  # Sadece son gun icin lh=24 yapilabilir.
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")
from seffaflik.elektrik import santraller, tuketim, uretim, yekdem
from seffaflik.elektrik.piyasalar import dengesizlik, dgp, genel, gip, gop, ia, yanhizmetler
first="2022-01-01"
last="2022-11-27"
UP = uretim.kgup(baslangic_tarihi=first, bitis_tarihi=last)
UP.columns=['Tarih', 'Saat', 'Doğalgaz', 'Barajlı', 'Linyit', 'Akarsu',
       'İthal Kömür', 'Rüzgar', 'Fuel Oil', 'Jeo Termal', 'Taş Kömür',
       'Biyokütle', 'Nafta', 'Diğer', 'KGUP']
YTP = tuketim.tahmin(baslangic_tarihi=first, bitis_tarihi=last)
YTP.columns=['Tarih',	'Saat',	'YTP']
CP = tuketim.gerceklesen(baslangic_tarihi=first, bitis_tarihi=last)
CP.columns=['Tarih',	'Saat',	'Consumption']
PTF = gop.ptf(baslangic_tarihi=first, bitis_tarihi=last)
PTF.columns=['Tarih',	'Saat',	'PtfTl',	'PtfEur','PtfUsd']
import pandas as pd
UP['Date'] = pd.to_datetime(UP.Tarih.astype(str)+' '+UP.Saat.astype(str)+':00')
UP['Date'] = pd.to_datetime(UP['Date'], format='%Y-%m-%dT%H:%M:%S')
CP['Date'] = pd.to_datetime(CP.Tarih.astype(str)+' '+CP.Saat.astype(str)+':00')
CP['Date'] = pd.to_datetime(CP['Date'], format='%Y-%m-%dT%H:%M:%S')
YTP['Date'] = pd.to_datetime(YTP.Tarih.astype(str)+' '+YTP.Saat.astype(str)+':00')
YTP['Date'] = pd.to_datetime(YTP['Date'], format='%Y-%m-%dT%H:%M:%S')
PTF['Date'] = pd.to_datetime(PTF.Tarih.astype(str)+' '+PTF.Saat.astype(str)+':00')
PTF['Date'] = pd.to_datetime(PTF['Date'], format='%Y-%m-%dT%H:%M:%S')

import pandasql as ps
import numpy as np
from datetime import date
import holidays
te=pd.merge(left=UP,right=YTP,left_on="Date",right_on="Date")
tf=pd.merge(left=te,right=PTF,left_on="Date",right_on="Date")
tf['IsHoliday'] = np.array( 1 if row.Date in holidays.TR() else 0 for index, row in tf.iterrows())
dx=ps.sqldf("select tf.*,CP.Consumption from tf LEFT JOIN CP ON tf.Date= CP.Date")
tx=dx[['Date','PtfUsd','YTP','KGUP','Consumption','IsHoliday']]

df=tx
import time
from datetime import datetime
from datetime import date
from sklearn.metrics import mean_absolute_error
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Hour']=df['Date'].dt.hour
df['WeekDay']=df['Date'].dt.dayofweek
df['HoW']=df['WeekDay']*24+df['Hour']
#df['Consumption2']=np.where(df['IsHoliday']==1,df['Consumption'].shift(168),df['Consumption'])
df['WLag1']  = df.Consumption.shift(24)
df['WLag2']  = df.Consumption.shift(24*2)
df['WLag3']  = df.Consumption.shift(24*3)
df['WLag4']  = df.Consumption.shift(24*4)
df['WLag7']  = df.Consumption.shift(24*7)
df['PtfUsd']  = df.PtfUsd
df['OffDay']=np.where ((df['WeekDay']==6) | (df['IsHoliday']==1),1,0)


bs=df['WLag1'].isnull().sum()
df['WLag1a']  = np.where(df['Hour']<24-bs,df.WLag1 ,df.WLag2)
df['ErrK']=df['Consumption']-df['KGUP']
df['ErrK1']=df.ErrK.shift(24)
df['ErrK2']=df.ErrK.shift(48)
df['ErrK1a']  = np.where(df['Hour']<24-bs,df.ErrK1 ,df.ErrK2)

df2=df[24*7:]
df2['CBR']=1


#Son lh saat haric train datasi olarak kullan
#NA lari olan rowlari veriden cikar Consumption gibi 
df2A= df2[:-lh]
df2A=df2A.dropna()
#Son lh saat test datasi olarak kullan eger kgup verisi
#yoksa hicbir data olmasin. Tahmin yapma
df3=df2[-lh:]
df3=df3.loc[df3['KGUP'].notnull()]
params={ 'max_depth': 2}
fs=-24*6*30
k=0
k1=k
k2=k+24
X_train2=df2A[['Hour','HoW','WLag1a','WLag2','WLag7','OffDay','YTP','KGUP','PtfUsd','ErrK1a']][fs:]
y_train2=df2A['Consumption'][fs:]
X_train2A=df2A[['Consumption','Hour','HoW','WLag1a','WLag2','WLag7','OffDay','YTP','KGUP','PtfUsd','ErrK1a']][fs:]
X_test2=df3[['Hour','HoW', 'WLag1a','WLag2','WLag7','OffDay','YTP','KGUP','PtfUsd','ErrK1a']][-lh:]
#y_test2=df2['Consumption'][k1:k2]
import catboost as cb
regressor_CBR =  cb.CatBoostRegressor(random_state=17,logging_level='Silent',**params)
#regressor_CBR =  MLPRegressor(random_state=17)
regressor_CBR.fit(X_train2,y_train2)
test_predictionCBR=regressor_CBR.predict(X_test2)
df3.loc[-lh: ,'CBR']=test_predictionCBR
#df3['CBR'][-lh:]=test_predictionCBR
#lh=240 icin tahmin  cizdirilmistir.MAPE hesaplanmistir.
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape
print('Modelin MAPEsi')
print(MAPE(df3['Consumption'],df3['CBR']))
print('Yuk Tahmin Planin MAPEsi')
print(MAPE(df3['Consumption'],df3['YTP']))
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x = df3.Date, y = df3.Consumption, name = "Gercek", line_color = 'blue'))
fig.add_trace(go.Scatter(x = df3.Date, y = df3.CBR, name = "Tahmin", line_color = 'red'))


#fig.add_trace(go.Scatter(x = sdge2.index, y = sdge2.Holiday*10, name = "Holiday",     line_color = 'green'))
fig.update_layout(title_text = "Turkiye Gerçek Elektrik Tüketim ve  Tahmin Verileri ", xaxis_rangeslider_visible = True)
fig.show()



Modelin MAPEsi
1.6190970055337353
Yuk Tahmin Planin MAPEsi
1.6390189836377818


CPU times: user 6.3 s, sys: 245 ms, total: 6.54 s
Wall time: 20.4 s


In [24]:
!pip install --pre pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install pycaret
%%time
lh=240  # Sadece son gun icin lh=24 yapilabilir.
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")
from seffaflik.elektrik import santraller, tuketim, uretim, yekdem
from seffaflik.elektrik.piyasalar import dengesizlik, dgp, genel, gip, gop, ia, yanhizmetler
first="2022-01-01"
last="2022-11-27"
UP = uretim.kgup(baslangic_tarihi=first, bitis_tarihi=last)
UP.columns=['Tarih', 'Saat', 'Doğalgaz', 'Barajlı', 'Linyit', 'Akarsu',
       'İthal Kömür', 'Rüzgar', 'Fuel Oil', 'Jeo Termal', 'Taş Kömür',
       'Biyokütle', 'Nafta', 'Diğer', 'KGUP']
YTP = tuketim.tahmin(baslangic_tarihi=first, bitis_tarihi=last)
YTP.columns=['Tarih',	'Saat',	'YTP']
CP = tuketim.gerceklesen(baslangic_tarihi=first, bitis_tarihi=last)
CP.columns=['Tarih',	'Saat',	'Consumption']
PTF = gop.ptf(baslangic_tarihi=first, bitis_tarihi=last)
PTF.columns=['Tarih',	'Saat',	'PtfTl',	'PtfEur','PtfUsd']
import pandas as pd
UP['Date'] = pd.to_datetime(UP.Tarih.astype(str)+' '+UP.Saat.astype(str)+':00')
UP['Date'] = pd.to_datetime(UP['Date'], format='%Y-%m-%dT%H:%M:%S')
CP['Date'] = pd.to_datetime(CP.Tarih.astype(str)+' '+CP.Saat.astype(str)+':00')
CP['Date'] = pd.to_datetime(CP['Date'], format='%Y-%m-%dT%H:%M:%S')
YTP['Date'] = pd.to_datetime(YTP.Tarih.astype(str)+' '+YTP.Saat.astype(str)+':00')
YTP['Date'] = pd.to_datetime(YTP['Date'], format='%Y-%m-%dT%H:%M:%S')
PTF['Date'] = pd.to_datetime(PTF.Tarih.astype(str)+' '+PTF.Saat.astype(str)+':00')
PTF['Date'] = pd.to_datetime(PTF['Date'], format='%Y-%m-%dT%H:%M:%S')

import pandasql as ps
import numpy as np
from datetime import date
import holidays
te=pd.merge(left=UP,right=YTP,left_on="Date",right_on="Date")
tf=pd.merge(left=te,right=PTF,left_on="Date",right_on="Date")
tf['IsHoliday'] = np.array( 1 if row.Date in holidays.TR() else 0 for index, row in tf.iterrows())
dx=ps.sqldf("select tf.*,CP.Consumption from tf LEFT JOIN CP ON tf.Date= CP.Date")
tx=dx[['Date','PtfUsd','YTP','KGUP','Consumption','IsHoliday']]

df=tx
import time
from datetime import datetime
from datetime import date
from sklearn.metrics import mean_absolute_error
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Hour']=df['Date'].dt.hour
df['WeekDay']=df['Date'].dt.dayofweek
df['HoW']=df['WeekDay']*24+df['Hour']
#df['Consumption2']=np.where(df['IsHoliday']==1,df['Consumption'].shift(168),df['Consumption'])
df['WLag1']  = df.Consumption.shift(24)
df['WLag2']  = df.Consumption.shift(24*2)
df['WLag3']  = df.Consumption.shift(24*3)
df['WLag4']  = df.Consumption.shift(24*4)
df['WLag7']  = df.Consumption.shift(24*7)
df['PtfUsd']  = df.PtfUsd
df['OffDay']=np.where ((df['WeekDay']==6) | (df['IsHoliday']==1),1,0)


bs=df['WLag1'].isnull().sum()
df['WLag1a']  = np.where(df['Hour']<24-bs,df.WLag1 ,df.WLag2)
df['ErrK']=df['Consumption']-df['KGUP']
df['ErrK1']=df.ErrK.shift(24)
df['ErrK2']=df.ErrK.shift(48)
df['ErrK1a']  = np.where(df['Hour']<24-bs,df.ErrK1 ,df.ErrK2)

df2=df[24*7:]
df2['CBR']=1


#Son lh saat haric train datasi olarak kullan
#NA lari olan rowlari veriden cikar Consumption gibi 
df2A= df2[:-lh]
df2A=df2A.dropna()
#Son lh saat test datasi olarak kullan eger kgup verisi
#yoksa hicbir data olmasin. Tahmin yapma
df3=df2[-lh:]
df3=df3.loc[df3['KGUP'].notnull()]
params={ 'max_depth': 2}
fs=-24*6*30
k=0
k1=k
k2=k+24
X_train2=df2A[['Hour','HoW','WLag1a','WLag2','WLag7','OffDay','YTP','KGUP','PtfUsd','ErrK1a']][fs:]
y_train2=df2A['Consumption'][fs:]
X_train2A=df2A[['Consumption','Hour','HoW','WLag1a','WLag2','WLag7','OffDay','YTP','KGUP','PtfUsd','ErrK1a']][fs:]
X_test2=df3[['Hour','HoW', 'WLag1a','WLag2','WLag7','OffDay','YTP','KGUP','PtfUsd','ErrK1a']][-lh:]
from pycaret.regression import *
s = setup(data = X_train2A, target = 'Consumption', session_id=123)
best = compare_models()
unseen_predictions = predict_model(best, data=X_test2)
df3['CBR']=1
df3.loc[-lh: ,'CBR']=unseen_predictions['prediction_label'].values
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape
print('Modelin MAPEsi')
print(MAPE(df3['Consumption'],df3['CBR']))
print('Yuk Tahmin Planin MAPEsi')
print(MAPE(df3['Consumption'],df3['YTP']))
#lh=240 icin tahmin  cizdirilmistir.MAPE hesaplanmistir.
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x = df3.Date, y = df3.Consumption, name = "Gercek", line_color = 'blue'))
fig.add_trace(go.Scatter(x = df3.Date, y = df3.CBR, name = "Tahmin", line_color = 'red'))


#fig.add_trace(go.Scatter(x = sdge2.index, y = sdge2.Holiday*10, name = "Holiday",     line_color = 'green'))
fig.update_layout(title_text = "Turkiye Gerçek Elektrik Tüketim ve  Tahmin Verileri ", xaxis_rangeslider_visible = True)
fig.show()